# Multi-turn


## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-5-Sonnet': 'anthropic.claude-3-5-sonnet-20240620-v1:0',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Opus': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'T

## 2. Claude-v3.5 모델 로딩

### LLM 로딩 (Claude-v3.5)

In [5]:
from langchain_aws import ChatBedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
llm_text = ChatBedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-Sonnet"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"],
    },
    #streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
llm_text

ChatBedrock(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f50937d3580>], client=<botocore.client.BedrockRuntime object at 0x7f506bc8c640>, model_id='anthropic.claude-3-5-sonnet-20240620-v1:0', model_kwargs={'max_tokens': 1024, 'stop_sequences': ['\n\nHuman']})

## How to use multi-turn

In [7]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages.ai import AIMessage
from langchain.schema.output_parser import StrOutputParser
from langchain_core.tracers import ConsoleCallbackHandler

In [8]:
chat_history = []

system_prompt = SystemMessagePromptTemplate.from_template("너는 질문에 대답하는 챗봇이야")
human_prompt = HumanMessagePromptTemplate.from_template("{question}")

prompt = ChatPromptTemplate([
    system_prompt,
    ("placeholder", "{chat_history}"),
    human_prompt
])


In [9]:
chain = prompt | llm_text | StrOutputParser()

In [11]:
question = "내 이름이 뭐라고?"
stream = chain.stream(
    {
        "chat_history": chat_history,
        "question": question
    },
    config={'callbacks': [ConsoleCallbackHandler()]}
        
)
res = ""
for chunk in stream: res += chunk

message = ("human", question)
chat_history.append(message)
message = ("ai", res)
chat_history.append(message)
chat_history



    

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "chat_history": [
    [
      "human",
      "안녕 나는 장동진이야"
    ],
    [
      "ai",
      "안녕하세요, 장동진님! 반갑습니다. 무엇을 도와드릴까요? 궁금한 점이 있으시면 언제든 물어보세요."
    ]
  ],
  "question": "내 이름이 뭐라고?"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatBedrock] Entering LLM run with input:
{
  "prompts": [
    "System: 너는 질문에 대답하는 챗봇이야\nHuman: 안녕 나는 장동진이야\nAI: 안녕하세요, 장동진님! 반갑습니다. 무엇을 도와드릴까요? 궁금한 점이 있으시면 언제든 물어보세요.\nHuman: 내 이름이 뭐라고?"
  ]
}
[chain/start] [chain:RunnableSequence > parser:StrOutputParser] Entering Parser run with input:
{
  "input": ""
}
당신의 이름은 장동진이라고 하셨습니다.[llm/end] [chain:RunnableSequence > llm:ChatBedrock] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "당신의

[('human', '안녕 나는 장동진이야'),
 ('ai', '안녕하세요, 장동진님! 반갑습니다. 무엇을 도와드릴까요? 궁금한 점이 있으시면 언제든 물어보세요.'),
 ('human', '내 이름이 뭐라고?'),
 ('ai', '당신의 이름은 장동진이라고 하셨습니다.')]

In [ ]:
이걸 중국어로 바꿔